In [ ]:
import gc
import glob
import os
import sys
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from random import randrange
from tqdm import tqdm, tnrange
from tqdm.notebook import tqdm_notebook

import warnings

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from keras.optimizer_v2.gradient_descent import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Lambda, Dropout, LeakyReLU, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from mlxtend.evaluate import accuracy

from IPython.display import clear_output

from tensorflow.keras.optimizers import Adam,RMSprop,SGD, Nadam
#from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignore tensorflow warnings
warnings.filterwarnings("ignore")

In [ ]:
path_data = "data/data_palm_vein/NIR"

In [ ]:
epochs = 20
batch = 32

print("\n ================= Training parameters ================= \n")
print("            Epochs :  {}   |   Batch size : {} ".format(epochs, batch))
print("\n ======================================================= \n")

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch,
    image_sizecallbacks=8, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.15,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
def create_model(model_tf, input_shape=(128, 128, 3), classes=500):
    if model_tf == 'resnet150v2': 
        model_transferlearning = ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_tf == 'resnet50v2':
        model_transferlearning = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_tf == 'vgg16':
        model_transferlearning = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_tf == 'vgg19':
        model_transferlearning = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_tf == 'inceptionresnetv2':
        model_transferlearning = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_tf == 'xception':
        model_transferlearning = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    model_transferlearning.tbatch_sizenable = False
    model = Sequential()
    model.add(model_transferlearning)
    model.add(Flatten())
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation='LeakyReLU'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
    
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.7, 
                                            min_lr=0.00000000001)
    return model, learning_rate_reduction
    

In [ ]:
#list_models = ["resnet150v2", "resnet50v2","vgg16", "vgg19", "inceptionresnetv2", "xception"]

list_models = ["resnet150v2", "resnet50v2","vgg16", "vgg19"]
list_accuracy = []
list_loss = []

In [ ]:
for model_tf in list_models:
    #clear_output(wait=True)
    model, learning_rate_reduction = create_model(model_tf)
    trained = model.fit(train_ds, validation_data = val_ds, epochs=epochs, batch_size=batch, callbacks=learning_rate_reduction)
    val = model.evaluate(test_ds, verbose=2)
    acc = val[1] * 100
    loss = val[0] * 100
    list_accuracy.append(acc)
    list_loss.append(loss)
#    with open('/trainHistoryDict', 'wb') as file_pi:
#        pickle.dump(trained.history, file_pi)


In [ ]:
df = pd.DataFrame({'Models': list_models, 'ACCURACY (%)': list_accuracy, 'LOSS (%)': list_loss})
df = df.sort_values(by=['ACCURACY (%)'])
df = df.reset_index(drop=True)
#df = df.set_title("Epochs :  {} | Batch_size : {} | Optimizer : Nadam".format(epochs, batch))
df

In [ ]:
fig = plt.figure(figsize = (8, 2))
ax = fig.add_subplot(111)

ax.table(cellText = df.values,
          rowLabels = df.index,
          colLabels = df.columns,
          loc = "center"
         )
ax.set_title("Epochs :  {} | Batch_size : {} | Optimizer : Nadam".format(epochs, batch))

ax.axis("off");


In [ ]:
trace1 = go.Bar(x = df.iloc[:,0].tolist(), y = df.iloc[:,1].tolist())

data1 = [trace1]
layout1 = go.Layout(
    title='Comparison of the Learning Methods',
    xaxis=dict(titlefont=dict(size=16)),
    yaxis=dict(title='ACCURACY (%)',gridwidth=1, gridcolor='#bdbdbd', range=[89, 99]),
    font=dict(size=16),
    bargap = 0.7,
    barmode='group')

fig = go.Figure(data=data1, layout=layout1)
py.iplot(fig, filename='grouped-bar')